# Likelihood for inferring key

Here I define some likelihood-based methods to infer a key for a cipher.


## Likelihoods based on single letter frequencies

The frequecy tables generated in the counting notebook are put to use here. First I look at the frequency of each letter. In this example, I encode a text using a caesar cipher (shift cipher) with a key of 14, and then return the most probably answer based on the likelihood of each letter.

In [1]:
from utils import *

In [2]:
bible_letters = make_letters('../data/bible.txt')
bible_letter_count = count_letters(bible_letters)
bible_letter_percent = normalize_counts_no_spaces(bible_letter_count)

The simple likelihood computation multiplies together the letter frequencies.
So for example, the likelihood for "the" will be the frequency of t times frequency of h
times frequency of e. 


In [4]:
print(find_likelihood("the",bible_letter_percent))
print(bible_letter_percent[alpha_list.index("t")] * \
      bible_letter_percent[alpha_list.index("h")] * \
      bible_letter_percent[alpha_list.index("e")])

0.001089168502310533
0.001089168502310533


And we can also compute the log-likelihood as the sum of the log frequencies.
This stops numerical error from the number getting tiny.

In [5]:
#need some examples of computing likelihoods
print(log(find_likelihood("the",bible_letter_percent)))
print(find_log_likelihood("the",bible_letter_percent))

-6.82234071576998
-6.822340715769981


When computing the likelihood, I ignore spaces. This is because spaces are often left unchanged when a cipher is encoded. See how decoding 'the' and 't h e' return the same result.

In [3]:
print(find_log_likelihood('the', bible_letter_percent))
print(find_log_likelihood('t h e', bible_letter_percent))

-6.822340715769981
-6.822340715769981


I also implemented a log-likelihood based on first counting the letter frequencies in the ciphertext and then computing the likelihood for any given key. See how this function returns the same answer as those in the previous cell.

In [4]:
text_letter_counts = count_letters('the')
num_key = []
for i in range(26):
    num_key.append(i)
print(compute_key_log_likelihood_singles(text_letter_counts, bible_letter_percent, num_key))

-6.822340715769981


Notice how the you now have two ways to change the outcome of the function: changing the text and changing the key. If you change the key at the index for 'e' and the index for 's', you get the equivalent of finding the log-likelihood of 'ths'.

In [6]:
num_key = []
for i in range(26):
    num_key.append(i)
num_key[18], num_key[4] = num_key[4], num_key[18]
print(compute_key_log_likelihood_singles(text_letter_counts, bible_letter_percent, num_key))
print(find_log_likelihood('ths', bible_letter_percent))

-7.596783010072889
-7.596783010072889


## Likelihood based on pair frequencies

Of course letters in English are not independent. Here I compute a likelihood based on the transitions between letters (a "Markov chain"). First I compute the transition matrix.

In [3]:
bible_pair_counts = count_letter_pairs(bible_letters)
bible_matrix = compute_transition_matrix(bible_pair_counts, 0.5)

The log-likelihood is based on the sum of the logarithms of the transition probabilities for all letter pairs. The first letter is taken care of by saying it is a pair of space and the letter.

In [5]:
print(find_pair_log_likelihood("the", bible_matrix))
print(log(bible_matrix[alpha_list.index(" ")][alpha_list.index("t")]) + \
      log(bible_matrix[alpha_list.index("t")][alpha_list.index("h")]) + \
      log(bible_matrix[alpha_list.index("h")][alpha_list.index("e")]))

-3.0931575455336557
-3.0931575455336557


In [4]:
text_pair_counts = count_letter_pairs('the')
num_key = []
for i in range(27):
    num_key.append(i)
print(compute_key_log_likelihood_pairs(text_pair_counts, bible_matrix, num_key))

-3.093157545533656


In [5]:
total = 0
for i in range(27):
    total += sum(text_pair_counts[i])
print(total)

3
